In [1]:
%cd /data/soyeonhong/nlq/nlq_lightning
%load_ext autoreload
%autoreload 2

/data/soyeonhong/nlq/nlq_lightning


In [2]:
from pathlib import Path
import pandas as pd
import json
import torch

In [5]:
p_egoclip = Path('data/egoclip.csv')

df = pd.read_csv(p_egoclip, delimiter='\t', on_bad_lines='skip')

p_ann = Path('/data/soyeonhong/nlq/nlq_lightning/data/nlq_v2/nlq_train.json')
anns = json.loads(p_ann.read_text())['videos']
nlq_video_id_lists = [ann['video_uid'] for ann in anns]        
df_new = df[df['video_uid'].isin(nlq_video_id_lists)]
df_new.head()

,video_uid,video_dur,narration_source,narration_ind,narration_time,clip_start,clip_end,clip_text,tag_verb,tag_noun,Unnamed: 10
2268,0031d268-818c-4ec4-a804-935be610a61a,1987.933333,narration_pass_1,1,11.7545,11.353754,12.155326,#C C carries a piece of tile with his hands.,[50],"[321, 517]",NaN
2269,0031d268-818c-4ec4-a804-935be610a61a,1987.933333,narration_pass_1,2,14.9555,14.554674,15.356246,#C C places the piece of tile on the floor.,[66],"[192, 517]",NaN
2270,0031d268-818c-4ec4-a804-935be610a61a,1987.933333,narration_pass_1,3,20.0674,19.666574,20.468146,#C C picks up a marker from the floor with his...,[93],"[192, 361, 321]",NaN
2271,0031d268-818c-4ec4-a804-935be610a61a,1987.933333,narration_pass_1,4,22.3433,21.942514,22.744086,#C C marks the piece of tile with the marker.,[46],"[361, 517]",NaN
2272,0031d268-818c-4ec4-a804-935be610a61a,1987.933333,narration_pass_1,5,25.8109,25.410104,26.211676,#C C moves the piece of tile on the floor with...,[50],"[192, 321, 517]",NaN


In [9]:
p_noun = Path('/data/datasets/ego4d_data/v2/annotations/narration_noun_taxonomy.csv')

df_noun = pd.read_csv(p_noun, delimiter=',', on_bad_lines='skip')

df_noun.head()

,label,group
0,ambulance,['ambulance']
1,apple,['apple']
2,apron,"['apron', 'drill bit', 'welding machine']"
3,arm,['arm']
4,"artwork_(art,_draw,_drawing,_painting,_sketch)","['art', 'artwork', 'draw', 'drawing', 'flower ..."


In [82]:
p_verb = Path('/data/datasets/ego4d_data/v2/annotations/narration_verb_taxonomy.csv')

df_verb = pd.read_csv(p_verb, delimiter=',', on_bad_lines='skip')

df_verb.head()

,label,group
0,"adjust_(regulate,_increase/reduce,_change)","['adjust', 'change', 'expand', 'increase/reduc..."
1,"apply_(spread,_smear)","['apply', 'plaster', 'smear', 'spread']"
2,"arrange_(straighten,_sort,_distribute,_align)","['align', 'arrange', 'disentangle', 'distribut..."
3,"attach_(plug-in,_join,_fasten,_connect,_attach)","['affix', 'attach', 'bind', 'binds', 'connect'..."
4,blow,['blow']


In [83]:
noun = list(set(df_noun['label'].values))

for idx, word in enumerate(noun):
    noun[idx] = word.split('_')[0]
    
noun = list(set(noun))


verb = list(set(df_verb['label'].values))

for idx, word in enumerate(verb):
    verb[idx] = word.split('_')[0]
    
verb = list(set(verb))

len(noun), len(verb)

(565, 116)

In [84]:
text_lists = list(set(df_new['clip_text']))

print(len(text_lists))
for i, text in enumerate(text_lists):
    text_lists[i] = text.replace('.', '')
    # if text_lists[i][-1] == ' ':
    #     text_lists[i] = ''
    
text_lists = list(set(text_lists))
len(text_lists)

NameError: name 'df_new' is not defined

In [3]:
from model.ours.egovlp.model import TextOnlyFrozenInTime

model = TextOnlyFrozenInTime(ckpt_path="/data/soyeonhong/nlq/nlq_lightning/data/egovlp-config-removed.pth").cuda().eval()
tokenizer = model.tokenizer

/data/soyeonhong/anaconda3/envs/groundvqa/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data/soyeonhong/anaconda3/envs/groundvqa/lib/python3.9/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/data/soyeonhong/anaconda3/envs/groundvqa/lib/python3.9/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/data/soyeonhong/anaconda3/envs/groundvqa/lib/python3.9/site-packages/huggingface_hub/file_download.py:115

In [85]:
tokens_noun = tokenizer(noun, padding=True, truncation=True, return_tensors='pt', max_length=512)
tokens_verb = tokenizer(verb, padding=True, truncation=True, return_tensors='pt', max_length=512)
with torch.no_grad():
    tokens = tokens_noun.to('cuda')
    memory_noun = model(tokens_noun)

with torch.no_grad():
    tokens = tokens_verb.to('cuda')
    memory_verb = model(tokens_verb)

In [86]:
memory_noun.shape, memory_verb.shape

(torch.Size([565, 256]), torch.Size([116, 256]))

In [78]:
import h5py
import os

video_features = h5py.File('/data/soyeonhong/nlq/nlq_lightning/data/unified/egovlp_internvideo.hdf5', 'r')

In [91]:
v_feat = video_features['00d9a297-d967-4d28-8e5a-6b891814ec65']
# v_feat = video_features['0aca0078-b6ab-41fb-9dc5-a70b8ad137b2']
# v_feat = video_features['1c597fc1-7bd0-4325-abbc-645e3ec71866']
v_feat = torch.from_numpy(v_feat[:, :256])
v_feat = v_feat.cuda()
v_feat = v_feat.unsqueeze(0)

In [92]:
k = 5
sim = torch.bmm(memory_noun.unsqueeze(0), v_feat.transpose(1, 2))
max_indices = torch.argmax(sim, dim=1)
topk_values, topk_indices = torch.topk(sim, k, dim=1)

for idx in range(v_feat.shape[1]):
    # Extract the top 5 indices for the current position
    top_indices = topk_indices[0, :, idx].tolist()

    # Map the indices to corresponding nouns and print them
    top_nouns = [noun[i] for i in top_indices]
    print(f"{idx * 0.53:0.5f} ~ {(idx + 1) * 0.53:0.5f}")
    print(f"{top_nouns}\n")


0.00000 ~ 0.53000
['meat', 'bacon', 'flour', 'chopstick', 'bowl']

0.53000 ~ 1.06000
['meat', 'bacon', 'chopstick', 'bowl', 'flour']

1.06000 ~ 1.59000
['meat', 'bacon', 'chopstick', 'flour', 'bowl']

1.59000 ~ 2.12000
['meat', 'chopstick', 'bacon', 'bowl', 'flour']

2.12000 ~ 2.65000
['meat', 'bacon', 'flour', 'chopstick', 'bowl']

2.65000 ~ 3.18000
['chopstick', 'meat', 'bacon', 'bowl', 'flour']

3.18000 ~ 3.71000
['meat', 'bacon', 'flour', 'chopstick', 'bowl']

3.71000 ~ 4.24000
['flour', 'bacon', 'meat', 'chopstick', 'bowl']

4.24000 ~ 4.77000
['chopstick', 'meat', 'bacon', 'bowl', 'flour']

4.77000 ~ 5.30000
['bowl', 'bacon', 'flour', 'chopstick', 'meat']

5.30000 ~ 5.83000
['meat', 'bacon', 'flour', 'bowl', 'chopstick']

5.83000 ~ 6.36000
['meat', 'bacon', 'chopstick', 'flour', 'chicken']

6.36000 ~ 6.89000
['meat', 'flour', 'bacon', 'bowl', 'chopstick']

6.89000 ~ 7.42000
['bowl', 'chopstick', 'meat', 'bacon', 'flour']

7.42000 ~ 7.95000
['chopstick', 'meat', 'bacon', 'plate', '

In [94]:
k = 1
sim = torch.bmm(memory_verb.unsqueeze(0), v_feat.transpose(1, 2))
max_indices = torch.argmax(sim, dim=1)
topk_values, topk_indices = torch.topk(sim, k, dim=1)

for idx in range(v_feat.shape[1]):
    # Extract the top 5 indices for the current position
    top_indices = topk_indices[0, :, idx].tolist()

    # Map the indices to corresponding nouns and print them
    top_nouns = [verb[i] for i in top_indices]
    print(f"{idx * 0.53:0.5f} ~ {(idx + 1) * 0.53:0.5f}")
    print(f"{top_nouns}\n")

0.00000 ~ 0.53000
['mix']

0.53000 ~ 1.06000
['mix']

1.06000 ~ 1.59000
['mix']

1.59000 ~ 2.12000
['mix']

2.12000 ~ 2.65000
['mix']

2.65000 ~ 3.18000
['mix']

3.18000 ~ 3.71000
['mix']

3.71000 ~ 4.24000
['mix']

4.24000 ~ 4.77000
['mix']

4.77000 ~ 5.30000
['mix']

5.30000 ~ 5.83000
['mix']

5.83000 ~ 6.36000
['mix']

6.36000 ~ 6.89000
['mix']

6.89000 ~ 7.42000
['mix']

7.42000 ~ 7.95000
['mix']

7.95000 ~ 8.48000
['mix']

8.48000 ~ 9.01000
['fry']

9.01000 ~ 9.54000
['fry']

9.54000 ~ 10.07000
['fry']

10.07000 ~ 10.60000
['step']

10.60000 ~ 11.13000
['mix']

11.13000 ~ 11.66000
['mix']

11.66000 ~ 12.19000
['sieve']

12.19000 ~ 12.72000
['fry']

12.72000 ~ 13.25000
['draw']

13.25000 ~ 13.78000
['mix']

13.78000 ~ 14.31000
['push']

14.31000 ~ 14.84000
['fry']

14.84000 ~ 15.37000
['watch']

15.37000 ~ 15.90000
['roll']

15.90000 ~ 16.43000
['wave']

16.43000 ~ 16.96000
['fry']

16.96000 ~ 17.49000
['fry']

17.49000 ~ 18.02000
['lock']

18.02000 ~ 18.55000
['lock']

18.55000 ~ 